# ***scalar.py***

Il programma è strutturato principalmente in due classi:

* *ScalarModel*
* *TabularLearner*

## *ScalarModel*

* **Inizializzazione**: 
    ``` python
    def __init__(self, reward, draw=False)
    ```
    Questa classe modella l'ambiente cellulare e gestisce il comportamento delle cellule nel tempo. Il costruttore inizializza il modello con il tipo di ricompensa (*reward*) e se disegnare o meno il grafico dell'evoluzione delle cellule (*draw*).

* **Reset**: 
    ``` python
    def reset(self)
    ```
    La funzione *reset* reimposta lo stato dell'ambiente cellulare, inizializzando il tempo, la quantità di glucosio e ossigeno, e crea una popolazione di cellule sane e cancerose. Con la riga seguente crea una lista contenente una cellula cancerosa e 1000 cellule sane. Tutti gli oggetti "cellula", sono creati passando il parametro 0. PERCHE? COME MAI CREO SOLO UNA CELLULA CANCEROSA?
    ``` python
    self.cells = [CancerCell(0)] + [HealthyCell(0) for _ in range(1000)]
    ```

* **Ciclo delle cellule**:
    ``` python
    def cycle_cells(self):
        ...
    ```
    Questa funzione simula il ciclo di vita delle cellule, dove ogni cellula consuma risorse e può dividersi (mitosi) se le condizioni lo permettono.

    *   ``` python
        to_add = []
        ```
        La lista *to_add* viene utilizzata per raccogliere le nuove cellule che si formeranno durante il ciclo.

    *   ``` python
        for cell in random.sample(self.cells, count):
        ```
        La funzione itera su un campione casuale di cellule. *random.sample(self.cells, count)* restituisce un campione di count cellule dall'elenco delle cellule attuali.


    *   ``` python
        res = cell.cycle(self.glucose, count / 278 , self.oxygen)
        ```
        Ogni cellula esegue il proprio ciclo di vita chiamando il metodo *cycle* della cellula stessa. Questo metodo riceve come parametri la quantità di glucosio disponibile, una frazione del numero totale di cellule (*count / 278*), e la quantità di ossigeno disponibile. *res* è una lista che contiene: Il consumo di glucosio della cellula, Il consumo di glucosio della cellula e un terzo valore opzionale che indica se la cellula si è divisa (mitosi) e il tipo di cellula risultante (sana o cancerosa). *count / 278* sarebbe il numero di cellule vicine ma siccome in questo caso siamo in una dimensione assumiamo quel numero (ç).

    *   ``` python
        if len(res) > 2:
            if res[2] == 0:
                to_add.append(HealthyCell(0))
            elif res[2] == 1:
                to_add.append(CancerCell(0))
        ```
        Se *res* contiene più di due elementi, significa che la cellula si è divisa. Se *res[2]* è 0, una nuova cellula sana viene aggiunta alla lista to_add. Se *res[2]* è 1, una nuova cellula cancerosa viene aggiunta alla lista to_add.

    *   ``` python
        self.glucose -= res[0]
        self.oxygen -= res[1]
        ```

        La quantità di glucosio e ossigeno nell'ambiente viene ridotta in base al consumo delle cellule (*res[0]* e *res[1]* rispettivamente).
    
    *   ``` python
        self.cells = [cell for cell in self.cells if cell.alive] + to_add
        ```
        L'elenco delle cellule viene aggiornato per includere solo le cellule che sono ancora vive (*cell.alive*) e per aggiungere le nuove cellule formate durante il ciclo (*to_add*).

* **Riempimento delle risorse**:
    ``` python
    def fill_sources(self):
    ```
    La funzione fill_sources() aggiunge una quantità fissa di glucosio (13000) e ossigeno (450000) all'ambiente. Questo simula l'apporto costante di risorse necessarie per la sopravvivenza delle cellule.


* **Avanzamento del tempo**: 
    ``` python
    def go(self, ticks=1):
    ```
    La funzione è responsabile dell'avanzamento del tempo nell'ambiente cellulare e dell'aggiornamento dello stato delle risorse e delle cellule.

    *   *ticks*: Un intero che rappresenta il numero di unità di tempo (ore) per cui avanzare la simulazione. Il valore predefinito è 1

    * self.time += 1
    Ogni iterazione del ciclo incrementa la variabile time di 1, rappresentando un'ora che passa nella simulazione.
    
    *   ``` python
        self.cycle_cells()
        ```
        La funzione cycle_cells() aggiorna lo stato di ogni cellula nell'ambiente. Ogni cellula consuma risorse (glucosio e ossigeno) e può dividersi (mitosi) se le condizioni lo permettono. Le cellule morte vengono rimosse dalla lista self.cells e le nuove cellule vengono aggiunte.

    *   ``` python
        if self.draw_graph:
        ```
        Se l'attributo draw_graph è True, vengono aggiornati i dati per il grafico. I seguenti dati vengono aggiunti alle rispettive liste: *self.ticks*: La lista dei tempi (ore), *self.ccell_counts*: La lista dei conteggi delle cellule cancerose e  *self.hcell_counts*: La lista dei conteggi delle cellule sane.

* **Irradiazione**:
    ``` python
    def irradiate(self, dose):
    ```
    Applica una dose di radiazione a tutte le cellule, determinando la loro sopravvivenza o morte in base alla dose ricevuta.

* **Osservazione**:
    ``` python
    def observe(self):
        return HealthyCell.cell_count, CancerCell.cell_count
    ```
    Ritorna il conteggio delle cellule sane e cancerose.

* **Azione**:
    La funzione act applica una dose di radiazione (determinata dall'azione) e avanza il tempo di 24 unità, ritornando la ricompensa aggiustata in base alla dose e agli effetti sulle cellule. 24 unità corrispondono a 24 ore (ç)
    * Calcolo e Ritorno della Ricompensa
        ``` python
        return self.adjust_reward(dose, pre_ccell - post_ccell, pre_hcell - min(post_hcell, m_hcell))
        ```
        Vedi punto seguente

* **Aggiustamento della ricompensa**
    ``` python
    def adjust_reward(self, dose, ccell_killed, hcells_lost):
    ```
    Calcola la ricompensa in base alla dose somministrata, al numero di cellule cancerose uccise e al numero di cellule sane perse. Di seguito sono riportate le formule per le ricompense:

    * Se lo stato è terminale:
        * *Se self.end_type* è *'L'* o *'T'*:
        $$R = -1$$

        * Altrimenti:
            * Se *self.reward* è *'dose'*:
            $$R = -\frac{\text{dose}}{200} + 0.5 + \frac{\text{HealthyCell.cell\_count}}{4000}$$

            * Altrimenti:
            $$R = 0.5 + \frac{\text{HealthyCell.cell\_count}}{4000}$$

    * Se lo stato non è terminale: 
        * Se *self.reward* è *'dose'* oppure *'oar'*:
        $$R = -\frac{\text{dose}}{200} + \frac{\text{ccell\_killed} - 5.0 \times \text{hcells\_lost}}{100000}$$

        * Se *self.reward* è *'killed'*:
        $$R = \frac{\text{ccell\_killed} - \text{reward} \times \text{hcells\_lost}}{100000}$$


    Il fattore 5.0 indica che la perdita di cellule sane ha un peso negativo cinque volte maggiore rispetto all'uccisione delle cellule cancerose.
    
    La funzione *'killed'* corrisponde all’obiettivo di massimizzare il danno alle cellule cancerose riducendo al minimo il danno alle cellule sane,*'dose'* aggiunge un parametro, portando l’agente a trovare trattamenti con dosi totali più piccole, poiché gli effetti a lungo termine correlati a questa dose totale non sono modellati nella simulazione.

    Nota: quando leggi il paper, Killed and dose fa riferimento a *'dose'* nel codice mentre Kelled corrisponde a *'killed'*. (§ Controlla formule reward nel codice e nel paper. C'è una differenza nelle formule che può essere dovuta al fatto che questo codice corrisponde al caso scalare)

    
* **Stato terminale**: 
    ``` python
    def inTerminalState(self):
    ```
    Determina se l'ambiente è in uno stato terminale (vittoria, sconfitta, o tempo esaurito).

* **Disegno del grafico**: 
    ``` python
    def draw(self, title):
    ```
    Disegna un grafico dell'evoluzione delle cellule nel tempo.



## *TabularLearner*

Questa classe implementa l'algoritmo di Q-learning tabulare per ottimizzare il trattamento radioterapico.

* **Inizializzazione** 

    ``` python
    def __init__(self, env, cancer_cell_stages, healthy_cell_stages, actions, state_type):
    ```
    * cancer_cell_stages e healthy_cell_stages rappresentano iL numero di stages (in ciascuno stage ci sarà un determinato numero di cellule)

    * In questa parte vengono calcolati gi helper (*state_helper_hcells* e *state_helper_ccells*). Servono a determinare il modo in cui i conteggi continui di cellule sane e cancerose vengono convertiti in stati discreti nel contesto dell'algoritmo di Q-learning. In altre parole, queste variabili definiscono i confini degli intervalli o "stages" in cui il numero di cellule è suddiviso per creare uno spazio di stati gestibile e finito.
        ``` python
        if (state_type == 'o'): #log
	        self.state_helper_hcells = exp(log(3500.0) / (healthy_cell_stages - 2.0))
	        self.state_helper_ccells = exp(log(40000.0) / (cancer_cell_stages - 2.0))
        else: # lin
	        self.state_helper_hcells = 3500.0 / (healthy_cell_stages - 2.0)
	        self.state_helper_ccells = 40000.0 / (cancer_cell_stages - 2.0)
        ```
        Nel caso lineare divido il numero massimo(?) di cellule per il numero di stages ottenendo così il numero di cellule in ciascuno di essi. Nel caso logaritmico, divido il logaritmo del numero di cellule per il numero di stage. Il logaritmo fornisce come risultato l'esponente della base, cioè: $$log_ea = x$$ $$e^x = a$$ Siccome non abbiamo bisogno dell'esponente ma del numero di cellule che si incrementano ad ogni stage sucessivo, è necessario  convertire il logaritmo utilizzando l'esponenziale (inverso del logaritmo naturale).

        ***NOTA***: Il motivo per cui si sottrae 2 dal denominatore nel calcolo delle variabili è legato alla gestione degli stati discreti alle estremità dell'intervallo di conteggio delle cellule. Gli stati agli estremi (0 e massimo) sono spesso dedicati a rappresentare condizioni estreme, come assenza di cellule o massimo affollamento. Sottrarre 2 assicura che ci sia spazio sufficiente tra gli stati centrali, permettendo una più ampia copertura delle condizioni centrali. Inoltre, mantenere uno spazio adeguato tra gli stati intermedi è cruciale per garantire che l'algoritmo possa distinguere tra variazioni più piccole nel conteggio delle cellule, migliorando l'accuratezza delle decisioni dell'agente di reinforcement learning.

* ***Discretizzazione dello stato*** (Stato --> Numero di cellule sane o cancerose):
    ``` python
    def ccell_state(self, count):
        if (self.state_type == 'o'): # log (natural logatitm)
            return min(self.cancer_cell_stages - 1, int(ceil(log(CancerCell.cell_count + 1) / log(self.state_helper_ccells))))
        else:
            return min(self.cancer_cell_stages - 1, int(ceil(CancerCell.cell_count / self.state_helper_ccells)))


    def hcell_state(self, count):
        if (self.state_type == 'o'): # log
            return min(self.healthy_cell_stages - 1, ceil(log(max(HealthyCell.cell_count -8, 1) ) / log(self.state_helper_hcells)))
        else:
            return min(self.healthy_cell_stages - 1, int(ceil(max(HealthyCell.cell_count - 9, 0) / self.state_helper_hcells)))
    ``` 
    Queste funzioni prendono  il numero di cellule (cancerose e sane) e le mappa a uno stato discreto. Questo stato è un indice che viene utilizzato per accedere alla tabella Q. In che modo? Divindendo il numero di cellule che si vuole discretizzare per l'helper (numero di cellule in ogni stage). Se, per esempio, il numero rapporto è di due, l'indice per quello stato (numero di cellule) sarà 2 (ç). In particolare:
    * **ceil()**: Arrotonda per eccesso
    * **int()**: Trasforma il numero in un interno siccome abbiamo bisogno di un indice discretizzato per lo stato. 
    * **min()**: Si assicura che lo stato risultante non ecceda il massimo numero di stati e che non si sfori l'indice massimo per gli stati.

* ***Conversione dello stato***: 
    ``` python
    def convert(self, obs):
        return self.ccell_state(obs[1]), self.hcell_state(obs[0])
    ```

    Converte l'osservazione attuale in uno stato utilizzabile dal Q-learning:
    * *obs*: È una tupla o lista che rappresenta lo stato corrente dell'ambiente, nel formato (numero_di_cellule_sane, numero_di_cellule_cancerose). Viene ottenuta dall'ambiente attraverso la funzione observe()
    * *obs[0]* rappresenta il conteggio delle cellule sane, mentre *obs[1]* rappresenta il conteggio delle cellule cancerose

* ***Allenamento***: 
    ``` python
    def train(self, steps, alpha, epsilon, disc_factor):
    ```
    Allena l'agente per un numero specificato di passi, aggiornando la tabella Q in base alle esperienze raccolte. Durante l'addestramento, l'agente esplora l'ambiente, aggiornando la tabella Q per apprendere la politica ottimale per somministrare il trattamento (dose di radiazione) alle cellule cancerose.

    *   ``` python
        while steps > 0: 
        ```
        Il ciclo di addestramento continua fino a quando ci sono ancora passi di addestramento disponibili.

    *   ``` python
        while not self.env.inTerminalState() and steps > 0: 
        ```
        Questo ciclo interno continua fino a quando l'ambiente non raggiunge uno stato terminale (vittoria, sconfitta, o tempo esaurito) e ci sono ancora passi disponibili.

    * La matrice ***Q*** viene aggiornata nel seguente modo: 
        $$Q(s, a) = (1 - \alpha) \cdot Q(s, a) + \alpha \cdot \left( r + \gamma \cdot \max_{a'} Q(s', a') \right)$$
        Dove:
        * $Q(s, a)$ è il valore Q corrente per lo stato \( s \) e l'azione \( a \).
        * $\alpha$ è il tasso di apprendimento (learning rate), un valore compreso tra 0 e 1, che determina quanto il nuovo valore stimato influisce sul valore Q esistente.
        * $r$ è la ricompensa immediata ricevuta dopo aver eseguito l'azione \( a \) nello stato \( s \).
        * $\gamma$ è il fattore di sconto (discount factor), anch'esso compreso tra 0 e 1, che misura quanto l'agente valuta le ricompense future rispetto a quelle immediate.
        * $\max_{a'} Q(s', a')$ è il valore Q massimo per il nuovo stato \( s' \) considerando tutte le azioni possibili, rappresentando la stima del miglior valore futuro raggiungibile.

* ***Test***:
    ``` python
    def test(self, episodes, disc_factor, verbose=False, eval=False):
    ```
    Testa l'agente su un numero specificato di episodi, raccogliendo statistiche sulle performance.
    * Inizializzazione delle Variabili
        ``` python
        sum_scores = 0.0
        sum_error = 0.0
        lengths_arr = []
        fracs_arr = []
        doses_arr = []
        sum_w = 0
        survivals_arr = []
        ```
        * *sum_scores*: Accumula la somma delle ricompense totali ottenute in ogni episodio.
        * *sum_error*: Accumula l'errore quadratico medio per ogni episodio, utile per valutare la qualità della stima delle ricompense.
        * l*engths_arr, fracs_arr, doses_arr*: Liste per raccogliere statistiche sugli episodi, come la lunghezza, il numero di frazioni di dose somministrate e la dose totale somministrata.
        * *sum_w*: Conta il numero di episodi che terminano con una vittoria ('W').
        * *survivals_arr*: Raccoglie il tasso di sopravvivenza delle cellule sane per ogni episodio.
    * Esecuzione degli Episodi di Test:
        ``` python
        for _ in range(episodes):
            self.env.reset()
            sum_r = 0
            err = 0.0
            fracs = 0
            doses = 0
            time = 0
            init_hcell = HealthyCell.cell_count
        ```
        * *self.env.reset()*: Reimposta l'ambiente all'inizio di ogni episodio, garantendo che le condizioni iniziali siano sempre le stesse.
        * *sum_r*: Somma delle ricompense ottenute nell'episodio corrente.
        * *err*: Errore quadratico accumulato durante l'episodio.
        * *fracs*: Conta il numero di frazioni di dose applicate.
        * *doses*: Somma delle dosi somministrate.
        * *time*: Tiene traccia del tempo trascorso in ore (ogni iterazione incrementa di 24 ore).
        * *init_hcell*: Memorizza il numero iniziale di cellule sane.

    * Simulazione dell'Episodio

    * Raccolta delle Statistiche:
        * Errore quadratico medio
        $$err += (r + disc_factor * self.Q[new_state][action] - self.Q[state][action]) ** 2.0$$

    * Valutazione dell'Episodio

* ***Esecuzione***: 
``` python
def run(self, n_epochs, train_steps, test_steps, init_alpha, alpha_mult, init_epsilon, final_epsilon, disc_factor):
``` 
Esegue l'allenamento e il test dell'agente per un numero specificato di epoche, adattando i parametri di apprendimento.

* ***Salvataggio e caricamento della tabella Q***: 
``` python
def save_Q(self, name):
    np.save(name, self.Q, allow_pickle=False)

def load_Q(self, name):
    self.Q = np.load(name+'.npy', allow_pickle=False)
``` 
Salva e carica la tabella Q da un file.


